In [7]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
import ignite
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
import numpy as np
from tqdm import tqdm

In [14]:
device = torch . device ( "cuda:0" if torch . cuda . is_available () else "cpu" )

In [8]:
from torch.utils.data import Dataset
import os

In [50]:
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

class PlotsData():
    IMG_SIZE = 800
    downs_imgs = "./dataset/Train/downs"
    peaks_imgs = "./dataset/Train/peaks"
    rises_imgs = "./dataset/Train/rises"
    test_imgs = "./dataset/Test"
    LABELS = {downs_imgs: 0, peaks_imgs: 1, rises_imgs: 2}
    training_data = []
    
    downscount = 0
    peakscount = 0
    risescount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(3)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])
                        
                        if label == self.downs_imgs:
                            self.downscount += 1
                        elif label == self.peaks_imgs:
                            self.peakscount += 1
                        elif label == self. rises_imgs:
                            self.risescount += 1
                        
                    except Exception as e:
                        pass
                        #print(label, f, str(e))
                        
        np.random.shuffle(self.training_data)
        print('downs:',PlotsData.downscount)
        print('peaks:',PlotsData.peakscount)
        print('rises:',PlotsData.risescount)

if REBUILD_DATA:
    plotsdatarebuild = PlotsData()
    plotsdatarebuild.make_training_data() 

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 483.46it/s]

downs: 0
peaks: 0
rises: 0


In [16]:
from natsort import natsorted
from PIL import Image

class TrainDataset(Dataset):
    def __init__(self, m_dir, transform):
        
        # Set the loading directory
        self.m_dir = m_dir
        self.transform = transform
        
        downs_imgs = os.listdir(os.path.join(m_dir, 'downs'))
        peaks_imgs = os.listdir(os.path.join(m_dir, 'peaks'))
        rises_imgs = os.listdir(os.path.join(m_dir, 'rises'))
        
        dimg = natsorted(downs_imgs)
        pimg = natsorted(peaks_imgs)
        rimg = natsorted(rises_imgs)
        self.total_imgs = dimg + pimg + rimg
        
    
    def __len__(self):
        # Return the previously computed number of images
        return len(self.total_imgs)
    
    
    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
     
        # Use PIL for image loading
        image = Image.open(img_loc).convert("RGB")
        # Apply the transformations
        tensor_image = self.transform(image)
        return tensor_image

    

In [26]:
transform_dict = {
        'src': transforms.Compose(
        [transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ]),
        'tar': transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ])
}

batch_size = 128

In [28]:
dataset = TrainDataset(m_dir=".\\dataset\\Train", transform=transform_dict)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                                          drop_last=False, num_workers=4)
print(next(iter(data_loader)).shape)  # prints shape of image with single batch

RuntimeError: DataLoader worker (pid(s) 11940, 11580, 10420, 2924) exited unexpectedly

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
    
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [5]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(3): # три полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data  # X это батч свойств, y это батч целевых переменных.
        net.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net(X.view(-1,784))  # передаем выпрямленный батч
        loss = F.nll_loss(output, y)  # вычисляем функцию потерь
        loss.backward()  # передаем это значение назад по сети
        optimizer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!  

In [ ]:
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1,784))
        #print(output)
        for idx, i in enumerate(output):
            #print(torch.argmax(i), y[idx])
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
print("Accuracy: ", round(correct/total, 3))